# Lyric scraper and singer prediction

# Information

In this project, i have used web scraping to get the lyrics for two singers and at the end i have used a machine learning algorithm to predict the singer of a given lyric.

In [ ]:
import requests
import time
import datetime
from bs4 import BeautifulSoup
import csv
import pandas as pd

# Singers

## Taylor Swift

In [ ]:
URL_Tswift = 'https://www.lyrics.com/artist/Taylor-Swift/816977'

headers = {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/110.0"}

page = requests.get(URL_Tswift, headers = headers)

In [ ]:
page.status_code

In [ ]:
#Extract the html text data
page_html = page.text
#print(page_html)

In [ ]:
soup1 = BeautifulSoup(page_html, 'html.parser')
soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')
#print(soup2)

In [ ]:
songs= soup2.find_all(name='td', attrs={'class':'tal qx'})
#print(songs)

In [ ]:
type(songs)


In [ ]:
len(songs)

In [ ]:
#url_list = list()


for index, element in enumerate(songs):# use enumerate to get the index 
    a_tag = element.find(name="a")
    title = a_tag.get_text()
    url_song = a_tag['href']
    print("Index:", index)
    print("Names:", title)
    full_song_url = "https://www.lyrics.com" + url_song
    print(full_song_url)
    response = requests.get(full_song_url, headers=headers)
    html_current_url = response.text
    with open(title, 'w', encoding='UTF8') as file:
        file.write(html_current_url)
    time.sleep(1)
 

In [ ]:
import os


lyrics_data = []

filedir = "./Lyrics/TaylorSwift/"

for filename in os.listdir(filedir):
    file_path = filedir + filename
    with open(file_path, 'r', encoding='UTF8') as file:
        html_content = file.read()
    singer = 'TaylorSwift'    
    soup = BeautifulSoup(html_content, 'html.parser')
    lyrics_tags = soup.find_all('pre', {'id':'lyric-body-text'})
    for lyrics_tag in lyrics_tags:
        lyrics = lyrics_tag.get_text()
        lyrics_data.append({'Singer': singer, 'Lyrics': lyrics})
        
df_TSwift = pd.DataFrame(lyrics_data)
print(df_TSwift)        

In [ ]:
type(df_TSwift)

In [ ]:
df_TSwift.head()

# Adele

In [ ]:
URL_Adele = 'https://www.lyrics.com/artist/Adele/861756'

headers = {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/110.0"}

page = requests.get(URL_Adele, headers = headers)

In [ ]:
#Extract the html text data
page_html = page.text
#print(page_html)

In [ ]:
soup1 = BeautifulSoup(page_html, 'html.parser')
soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')
#print(soup2)

In [ ]:
songs= soup2.find_all(name='td', attrs={'class':'tal qx'})
#print(songs)

In [ ]:

folder_path = "./Lyrics/Adele/"

for index, element in enumerate(songs):# use enumerate to get the index 
    #print(index, element)
    a_tag = element.find(name="a")
    title = a_tag.get_text()
    url_song = a_tag['href']
    print("Index:", index)
    print("Names:", title)
    full_song_url = "https://www.lyrics.com" + url_song
    print(full_song_url)
    response = requests.get(full_song_url, headers=headers)
    html_current_url = response.text
    file_path = folder_path + f'{index}_{title}'
    with open(file_path, 'w', encoding='UTF8') as file:
        file.write(html_current_url)
    time.sleep(1)

In [ ]:

lyrics_data = []

filedir = "./Lyrics/Adele/"

for filename in os.listdir(filedir):
    file_path = filedir + filename
    with open(file_path, 'r', encoding='UTF8') as file:
        html_content = file.read()
    singer = 'Adele'    
    soup = BeautifulSoup(html_content, 'html.parser')
    lyrics_tags = soup.find_all('pre', {'id':'lyric-body-text'})
    for lyrics_tag in lyrics_tags:
        lyrics = lyrics_tag.get_text()
        lyrics_data.append({'Singer': singer, 'Lyrics': lyrics})
        
        
df_Adele = pd.DataFrame(lyrics_data)
print(df_Adele)     

In [ ]:
print(lyrics_data) 

In [ ]:
df_Adele.head()

In [ ]:
combined_df = pd.concat([df_TSwift, df_Adele], ignore_index=True)

In [ ]:
combined_df.head(1000)

# Bag of words

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
import string

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as skl_stopwords


lemmatizer = WordNetLemmatizer()
tokenizer= TreebankWordTokenizer()

nltk.download("wordnet")
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')

In [ ]:
# corpus
corpus = combined_df['Lyrics']#.tolist()

#labels
labels = combined_df['Singer']#.tolist()

In [ ]:
# This is a custom function to strip punctuation, tokenize and lemmatize as well as remove stopwords
def tokenize_lemmatize(text, stopwords=skl_stopwords, tokenizer=tokenizer, lemmatizer=lemmatizer):
    text = ''.join([ch for ch in text if ch not in string.punctuation]) #remove punctuation
    tokens = nltk.word_tokenize(text) 
    return [lemmatizer.lemmatize(token) for token in tokens if token not in stopwords] 
    #we have to filter for stopwprds at this stage, otherwise some stopwords get truncated and missed if passed to CountVectorizer

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=tokenize_lemmatize, stop_words=None)

In [ ]:
X_vec = vectorizer.fit_transform(corpus)
X_vec_df = pd.DataFrame(X_vec.todense(), columns=vectorizer.get_feature_names_out(), index=labels)

In [ ]:
X_vec_df

In [ ]:
X_vec_df.reset_index().rename(columns={'index': 'Singer'}, inplace=True)

In [ ]:
X_vec_df.reset_index(inplace=True)
X_vec_df.rename(columns={'index': 'Singer'}, inplace=True)


In [ ]:
X_vec_df.head()

In [ ]:
X_vec_df.columns

In [ ]:
x = X_vec_df.drop(['Singer'], axis=1)


y = X_vec_df['Singer']

In [ ]:
y.head()

In [ ]:
x.head()

In [ ]:
# classifiers
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB


In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, random_state=42, test_size = 0.2)

In [ ]:
model = MultinomialNB()

In [ ]:
model.fit(x_train, y_train)

In [ ]:
model.score(x_train, y_train)

In [ ]:
model.score(x_val, y_val)

In [ ]:
new_lyrics = ["Cause I′m not your princess, this ain't a fairytale"]



In [ ]:
new_vectors = vectorizer.transform(new_lyrics)

In [ ]:
new_vectors.todense()

In [ ]:
model.predict(new_vectors)

In [ ]:
model.predict_proba(new_vectors)